In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, Comment

In [2]:
def get_html_data(url):
    page = requests.get(url)
    return page

In [3]:
link = "https://www.pro-football-reference.com/years/"
years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
years_SO1 = ["2015", "2016", "2017"]
years_SO2 = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014"]
Team_Offense = ["Team", "Games_Played", "Points_Scored", "Yards", "Off_Plays", "Yards_per_OffPlay", "Turnover_Losts", "Fumble_Lost", "First_Down", "Pass_Complete", "Pass_Attempt", "Pass_Yards", "Pass_TD", "Inter_Thrown", "Yards_Gain_per_Pass_Attempt", "Pass_First_Down", "Rush_Attempts", "Rush_Yards", "Rush_TD", "Yards_Gain_Per_Rush", "Rush_First_Down", "Penalties_Commited", "Penalties_Yards", "Penalties_First_Down", "Per_Off_Score", "Per_Off_Turnover", "Expected_Points"]
Passing_Offense = ["Team", "Games_Played", "Passes_Complete", "Passes_Attempt", "Pers_Pass_Complete", "Yards_Passes", "Pass_TD", "Pers_TD_Attempt_Pass", "Interc_Thrown", "Pers_Interc_Attempt_Pass", "Longest_Pass_Complete", "Yards_per_Pass_Attempt", "Adj_Yards_per_Pass_Attempt", "Yards_per_Pass_Complete", "Yards_per_Game", "QB_Rate", "Times_Sacked", "Yards_Lost_Sacks", "Net_Yards_Gain_Pass_Attempt", "Adj_Net_Yards_Gain_Pass_Attempt", "Pers_Sacks_Attempt_Pass", "Comebacks_by_QB", "Win_Drive_by_QB", "Expect_Points_Pass_Offense"]
Rushing_Offense = ["Team", "Games_Played", "Rush_Attmepts", "Yards_Rush", "Rush_TD", "Longest_Rush", "Yards_per_Attempt_Rush", "Rush_Yards_per_Game", "Number_Fumbles", "Expect_Points_Rush_Offense"]
Kick_Punt_Return = ["Team", "Games_Played", "Punt_Return", "Punt_Yardage", "Punt_Return_TD", "Longest_Punt_Return", "Yards_per_Punt_Return", "Kickoff_Return", "Kickoff_Yardage", "Kickoff_Return_TD", "Longest_Kickoff_Return", "Yards_per_Kickoff_Return", "All_Purpose_Yards"]
Kicking_Punting = ["Team", "Games_Played", "FGA_0_19", "FGM_0_19", "FGA_20_29", "FGM_20_29", "FGA_30_39", "FGM_30_39", "FGA_40_49", "FGM_40_49", "FGA_50_", "FGM_50_", "FGA", "FGM", "FG_Perc", "Extra_Point_Attempt", "Extra_Point_Made", "Extra_Point_Perc", "Times_Punted", "Punt_Yardage", "Longest_Punt", "Times_Punt_Bloacked", "Yards_per_Punt"]
Scoring_Offense_2014 = ["Team", "Games_Played", "RshTD", "RecTD", "PR_TD", "KR_TD", "FblTD", "IntTD", "Other_TD", "All_TD", "2_Point_Conv_Made", "2_Point_Conv_Attmept", "Extra_Point_Made", "Extra_Point_Attempt", "FGM", "FGA", "Safties_Team", "Total_Points", "Point_per_Game"]
Scoring_Offense_2015_2017 = ["Team", "Games_Played", "RshTD", "RecTD", "PR_TD", "KR_TD", "FblTD", "IntTD", "Other_TD", "All_TD", "2_Point_Conv_Made", "2_Point_Conv_Attmept", "Defence_2_Point_Conv", "Extra_Point_Made", "Extra_Point_Attempt", "FGM", "FGA", "Safties_Team", "Total_Points", "Point_per_Game"]
Conversions = ["Team", "Games_Played", "3_Down_Attempts", "3_Down_Conv", "3_Down_Conv_Perc", "4_Down_Attempts", "4_Down_Conv", "4_Down_Conv_Perc", "Red_Zone_Attempt", "Red_Zone_Score", "Red_Zone_Score_Perc"]
Drive_Average = ["Team", "Games_Played", "Num_Drives", "Playes_Drive", "Perc_Drive_Off_Score", "Perc_Drive_Off_Turnover", "Avg_Plays_per_Drive", "Yards_per_Drive", "Avg_Start_Pos", "Avg_Time_Drive", "Avg_Points_per_Drive"]

In [4]:
def get_teams_data(page):
    x = BeautifulSoup(page.content, "html.parser")
    z = x.find("div", {"id":"site_menu"})
    t = z.find_all("li")
    teams = t[2].find_all("a")
    return teams

In [5]:
def extract_teams(teams):
    link = [t1.attrs['href'].split("/") for t1 in teams]
    del link[0]
    team = [t2.get_text() for t2 in teams]
    del team[0]
    link1 = [l[1] for l in link]
    link2 = [l[2] for l in link]
    link3 = [l[3] for l in link]
    team_data = pd.DataFrame({
                "Team": team,
                "Link1": link1,
                "Link2": link2,
                "Link3": link3})
    return team_data

In [6]:
def extract_html_data(page, div_id):
    x = BeautifulSoup(page.content, "html.parser")
    y = x.find("div", {"id": "content"})
    z = y.find("div", {"id": div_id})
    return z

In [7]:
def extract_table_data(div_data, tab_name):
    comments = div_data.findAll(text=lambda text:isinstance(text, Comment))
    x = comments[0].extract()
    y = BeautifulSoup(x, "html.parser")
    table = y.find("table", {"id": tab_name})
    return table

In [8]:
def extract_team_offense(table_data):
    tbody = table_data.find("tbody")
    tr = tbody.find_all("tr")
    data = pd.DataFrame(columns=Team_Offense)
    for row in tr:
        td = row.find_all("td")
        c = [col.get_text() for col in td]
        df = pd.DataFrame(np.array(c).reshape(1,27), columns=Team_Offense)
        data = data.append(df)
    return data

In [13]:
def extract_team_data(table_data, cols):
    tbody = table_data.find("tbody")
    tr = tbody.find_all("tr")
    data = pd.DataFrame(columns=cols)
    for row in tr:
        td = row.find_all("td")
        c = [col.get_text() for col in td]
        df = pd.DataFrame(np.array(c).reshape(1,len(cols)), columns=cols)
        data = data.append(df)
    return data

In [10]:
def final_data(cols, content, tab):
    data = pd.DataFrame(columns=cols)
    data["Year"] = ""
    for year in years:
        url = link + year + "/"
        page = get_html_data(url)
        div_data = extract_html_data(page, content)
        table_data = extract_table_data(div_data, tab)
        table = extract_team_data(table_data, cols)
        table["Year"] = year
        data = data.append(table)
        
    data = data.reset_index()
    data = data.drop(["index"], axis=1)
    return data

In [ ]:
"""Team_Offense_Data = pd.DataFrame(columns=Team_Offense)
Team_Offense_Data["Year"] = ""
for year in years:
    url = link + year + "/"
    page = get_html_data(url)
    div_data = extract_html_data(page, "all_passing")
    table_data = extract_table_data(div_data, "passing")
    table = extract_team_data(table_data, Team_Offense)
    table["Year"] = year
    Team_Offense_Data = Team_Offense_Data.append(table)"""

In [11]:
Team_Offense_Data = final_data(Team_Offense, "all_team_stats", "team_stats")
Team_Offense_Data.to_csv("Team_Offense.csv")

In [14]:
Passing_Offense_Data = final_data(Passing_Offense, "all_passing", "passing")
Passing_Offense_Data.to_csv("Passing_Offense.csv")

In [15]:
Rushing_Offense_Data = final_data(Rushing_Offense, "all_rushing", "rushing")
Rushing_Offense_Data.to_csv("Rushing_Offense.csv")

In [16]:
Kick_Punt_Return_Data = final_data(Kick_Punt_Return, "all_returns", "returns")
Kick_Punt_Return_Data.to_csv("Kick_Punt_Return.csv")

In [17]:
Kicking_Punting_Data = final_data(Kicking_Punting, "all_kicking", "kicking")
Kicking_Punting_Data.to_csv("Kicking_Punting.csv")

In [18]:
Scoring_Offense_Data = final_data(Scoring_Offense, "all_team_scoring", "team_scoring")
Scoring_Offense_Data.to_csv("Scoring_Offense.csv")

ValueError: cannot reshape array of size 19 into shape (1,20)

In [19]:
Conversions_Data = final_data(Conversions, "all_team_conversions", "team_conversions")
Conversions_Data.to_csv("Conversions.csv")

In [20]:
Drive_Average_Data = final_data(Drive_Average, "all_drives", "drives")
Drive_Average_Data.to_csv("Drive_Average.csv")